# CNN Project Information

## Informace

You can get 10 to 20 points

Every project must include brief description of the dataset

Number of images, number of classes and class balance
Examples of the images
What metrics scores you have decided to use

e.g. Accuracy, Precision, Recall, F1-score etc.
Also state which one of the scores is the most important from your point of view given the class balance, task, ...
Try minimally 2 different models

The first model should be built from scratch, i.e. create your own architecture and train the model
If you perform various experiments or parameters tuning be sure to include everyting in the Notebook step by step with some brief comments about the experiments (e.g. effect of BatchNorm/layer sizes/optimizer on accuracy/train time/...)
The second model will employ transfer learning techniques
You can try any model you wish, e.g. ResNet, Inception, MobileNet, etc.
Take a look at this list and try at least one of them
Fine tune the model for your dataset and compare it with the first one
Mandatory part of every project is a summary at the end in which you summarize the most interesting insight obtained.

Result is a Jupyter Notebook with descriptions included or a PDF report + source codes.

Deadline is 3. 4. 2022

## Příklady datových sad

- https://www.kaggle.com/joosthazelzet/lego-brick-images
- https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia
- https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
- https://www.kaggle.com/sanikamal/horses-or-humans-dataset
- https://www.kaggle.com/ihelon/lego-minifigures-classification

# Vypracování projektu

## Vybraná datová sada

- https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

## Informace ohledně datové sady

## Jakou metriku použijeme?

## Klasifikace

### Model 1

### Model 2

## Zhodnocení